In [ ]:
import torch as t
from torch import Tensor, nn
from torch.nn.functional import normalize
import transformer_lens
from transformer_lens import HookedTransformer
from dataclasses import dataclass
import math
import torch.nn.functional as F
import wandb
from sparse_QK_trainer import SparseQK, train_sparse_QK

In [ ]:
model = transformer_lens.HookedTransformer.from_pretrained("gpt2", fold_ln = True)

In [ ]:
batch_size = 20

import datasets
dataset = datasets.load_dataset("NeelNanda/pile-10k", split="train")
print(dataset)
print(dataset[0]['text'][:100])
tokens_dataset = transformer_lens.utils.tokenize_and_concatenate(dataset, model.tokenizer, streaming=False, max_length=20, column_name="text", add_bos_token=True, num_proc=4)
data_loader = t.utils.data.DataLoader(tokens_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
cfg = {"d_hidden": 4096,
       "l1_coeff": 1e-6,
       "d_model": 768,
       "n_heads": 1,
       "d_head": 64,
       "seed": 87,
       "device": "cuda:0",
       "dead_freq": 1e-7}

In [ ]:
sparse_model = train_sparse_QK(model, cfg, 1, 10, data_loader)